# KDD99 Decision Tree Intrusion Detection

### Importing Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import pandas as pd
from IPython.display import clear_output as clr
from sklearn.utils import class_weight
%matplotlib inline

In [2]:
import keras as kk
import keras.models as km
import keras.layers as kl
import keras.optimizers as ko
import keras.regularizers as kr
from sklearn.model_selection import train_test_split as tts

Using TensorFlow backend.


In [3]:
import tensorflow as tf

In [4]:
from sklearn.preprocessing import OneHotEncoder as OHE
from sklearn.preprocessing import LabelEncoder as LE

### Setting working Directory

In [5]:
dir_path = (os.getcwd()+'\\').replace("\\","/")
print("Working Directory : ", dir_path)

Working Directory :  /media/abhikcr/New Volume/Study/M.tech 1st Sem/Cyber Security/Homeworks/KDD/extracted/


### Fucntions to Load files..

In [6]:
def file_len(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1

In [7]:
def readlines(fname, start, end):
#     fname = 'kddcup.data.corrected'
    file = open(fname)
    batch = []
    r = end - start
    for i in range(start):
        line = file.readline()
    for i in range(r):
        line = file.readline()
        batch.append(line.split(','))
    file.close()
    return batch

In [8]:
def rmreadlines(fname, size, ntypes):
#     fname = 'kddcup.data.corrected'
    Nobs = file_len(fname)
    choices = np.sort(np.random.choice(Nobs,size, replace=False))
    flag = False
    k = 0
    
    file = open(fname)
    batch = []
    uniq = []
    
    for i in range(Nobs):

        line = file.readline()
        var = line.split(',')
        size = len(var)
        val = var[size-1]
        
        if(val not in uniq):
            batch.append(var)
            uniq.append(val)
            if(len(uniq) == ntypes):
                flag = True
        
        if(k<choices.shape[0]):
            if(choices[k] == i):
                batch.append(line.split(','))
                k = k+1
            else:
                pass
        else:
            if(flag):
                break;
    
    file.close()
    return batch

## Preprocessing Splitting data in train and test files

In [9]:
def split_file(fname, trainname, testname):
    Nobs = file_len(fname)
    tfp = int(Nobs*0.25)
    choices = np.sort(np.random.choice(Nobs,tfp, replace=False))
    k = 0
    file = open(fname)
    filetr = open(trainname, 'w')
    filets = open(testname, 'w')
    
    uniq_pro = []
    uniq_srv = []
    uniq_flg = []
    
    uniq = []
    
    for i in range(Nobs):
        
        line = file.readline()
        
        var = line.split(',')
        arr_small  = var[1:4]
        
        
        size = len(var)
        val = var[size-1]
        
        if(val in uniq):
            pass
        else:
            filets.write(line)
            uniq.append(val)
            
        if(arr_small[0] not in uniq_pro):
            uniq_pro.append(arr_small[0])
            
        if(arr_small[1] not in uniq_srv):
            uniq_srv.append(arr_small[1])
            
        if(arr_small[2] not in uniq_flg):
            uniq_flg.append(arr_small[2])
        
        
        if(k<choices.shape[0]):
            if(choices[k] == i):
                filets.write(line)
                k = k+1
            else:
                filetr.write(line)
        else:
            filetr.write(line)
            
    filets.close()
    filetr.close()
    file.close()
    return Nobs, choices.shape[0], [uniq_pro,uniq_srv, uniq_flg]

In [10]:
# Already spliited dont repeat...
split = False
if(split):
    Nobs, test_size, cat_uniqs = split_file('kddcup.data.corrected', 'train','test')

In [11]:
# np.save('uniqs',cat_uniqs)
cat_uniqs = list(np.load('uniqs.npy'))
cat_uniqs[0]

['tcp', 'udp', 'icmp']

In [12]:
len(cat_uniqs[1])

70

### Loading trainning data batchwise...

In [13]:
def myArrayConverter(arr):
    '''
    To convert string to floats
    '''
    convertArr = []
    for s in arr.ravel():    
        try:
            value = float(s)
        except ValueError:
            value = s
        convertArr.append(value)
    return array(convertArr,dtype=object).reshape(arr.shape)

In [14]:
# file names
trainfile = 'train'
testfile = 'test'

In [15]:
train_size = file_len(trainfile)
print('Training data has',train_size, 'observations')

Training data has 3673824 observations


In [16]:
batch_size = 100000
n_batches = int(np.ceil(train_size/batch_size))

### Bacth Processing...

Function to convert categorical Y values to one_hot vector

In [17]:
def one_hot(a, num_classes):
    return np.squeeze(np.eye(num_classes)[a.reshape(-1)])

Creating label ids

In [18]:
label_list = pd.read_csv('training_attack_types', delimiter = ' ', index_col = False, names =['type', 'category'])
label_list = label_list.assign(id_type=(label_list['type']).astype('category').cat.codes)
label_list = label_list.assign(id_cat=(label_list['category']).astype('category').cat.codes)
label_list = label_list.append({'type':'normal', 'category':'normal','id_type':'-1','id_cat':'-1'}, ignore_index = True)
label_list['id_type'] = label_list['id_type'].astype(int)+1
label_list['id_cat'] = label_list['id_cat'].astype(int)+1

In [19]:
nClasses = label_list.shape[0]
label_list

,type,category,id_type,id_cat
0,back,dos,1,1
1,buffer_overflow,u2r,2,4
2,ftp_write,r2l,3,3
3,guess_passwd,r2l,4,3
4,imap,r2l,5,3
5,ipsweep,probe,6,2
6,land,dos,7,1
7,loadmodule,u2r,8,4
8,multihop,r2l,9,3
9,neptune,dos,10,1


In [20]:
import gc

In [21]:
gc.collect()

0

In [22]:
red = False
if(red):
    df = pd.read_csv('train')

    df2 = df.drop_duplicates()

    df = None
    gc.collect()

    batcharr = df2.values

    print(batcharr.shape)

    np.save('train_red', df2.values)

    df = pd.read_csv('test')

    df2 = df.drop_duplicates()

    df = None
    gc.collect()

    batcharr = df2.values

    print(batcharr.shape)

    np.save('test_red', df2.values)

In [23]:
batcharr = np.load('train_red.npy')

X = batcharr[:,:-1]
Y = batcharr[:,-1]
Y_num = np.ones(Y.shape)
Y_num[Y == 'normal.\n'] = 0
Y_cat = Y_num.copy()
Y_type = Y_num.copy()
for i in label_list['type']:
    Y_cat[Y == i+str('.\n')] = label_list[label_list['type'] == i]['id_cat']
    Y_type[Y == i+str('.\n')] = label_list[label_list['type'] == i]['id_type']


In [24]:
# just to see class imbalance bacth wise....
labels = np.unique(Y , return_counts = True)
# Y.shape

In [25]:
Y_num = np.ones(Y.shape)
Y_num[Y == 'normal.\n'] = 0
Y_cat = Y_num.copy()
Y_type = Y_num.copy()

for i in label_list['type']:
    Y_cat[Y == i+str('.\n')] = label_list[label_list['type'] == i]['id_cat']
    Y_type[Y == i+str('.\n')] = label_list[label_list['type'] == i]['id_type']

Y_num_hot = one_hot(Y_num.astype(int), 2)
Y_cat_hot = one_hot(Y_cat.astype(int), 5)
Y_type_hot = one_hot(Y_type.astype(int), 23)

X_new_hot = np.delete(X,[1,2,3],1)

X_new_hot2 = X[:,[1,2,3]]

# X_new.shape

model_pro_le = LE()
model_pro_ohe = OHE(handle_unknown='ignore')
model_srv_le = LE()
model_srv_ohe = OHE(handle_unknown='ignore')
model_flg_le = LE()
model_flg_ohe = OHE(handle_unknown='ignore')

model_pro_ohe.fit_transform(model_pro_le.fit_transform(cat_uniqs[0]).reshape(-1,1))
model_srv_ohe.fit_transform(model_srv_le.fit_transform(cat_uniqs[1]).reshape(-1,1))
model_flg_ohe.fit_transform(model_flg_le.fit_transform(cat_uniqs[2]).reshape(-1,1))

X_pro = model_pro_ohe.transform(model_pro_le.transform(X_new_hot2[:,0]).reshape(-1,1)).todense()
X_srv = model_srv_ohe.transform(model_srv_le.transform(X_new_hot2[:,1]).reshape(-1,1)).todense()
X_flg = model_flg_ohe.transform(model_flg_le.transform(X_new_hot2[:,2]).reshape(-1,1)).todense()

# X_srv.shape

X_new = np.concatenate((X_new_hot , X_pro , X_srv, X_flg), axis = 1).astype(float)

X_new.shape

(833529, 122)

In [26]:
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn import tree

/home/abhikcr/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [27]:
from sklearn.metrics import confusion_matrix as cmat

In [28]:
from sklearn.metrics import precision_recall_fscore_support as prfs

In [29]:
model = tree.DecisionTreeClassifier()

In [30]:
model.fit(X_new,Y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [31]:
model.score(X_new, Y)

0.9999892025352447

In [32]:
Y_pr = model.predict(X_new)

In [46]:
np.set_printoptions(linewidth = 200)
print(cmat(Y, Y_pr))

[[   750      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0]
 [     0     24      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0]
 [     0      0      8      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0]
 [     0      0      0     38      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0]
 [     0      0      0      0      9      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0]
 [     0      0      0      0      0   2996      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0]
 [     0      0 

In [47]:
labels = np.unique(Y)
labels

array(['back.', 'buffer_overflow.', 'ftp_write.', 'guess_passwd.', 'imap.', 'ipsweep.', 'land.', 'loadmodule.', 'multihop.', 'neptune.', 'nmap.', 'normal.', 'perl.', 'phf.', 'pod.', 'portsweep.',
       'rootkit.', 'satan.', 'smurf.', 'spy.', 'teardrop.', 'warezclient.', 'warezmaster.'], dtype=object)

In [48]:
prfsarr = np.array(prfs(Y, Y_pr, labels = labels))

In [49]:
pd.DataFrame(prfsarr.T, columns = ['precison', 'recall', 'f_score', 'support'], index = labels).sort_values(by = 'support', ascending = False)

,precison,recall,f_score,support
normal.,0.999997,0.999997,0.999997,616654.0
neptune.,1.000000,1.000000,1.000000,201040.0
satan.,1.000000,1.000000,1.000000,4081.0
ipsweep.,0.998334,1.000000,0.999166,2996.0
portsweep.,1.000000,0.999640,0.999820,2779.0
smurf.,1.000000,1.000000,1.000000,2397.0
nmap.,1.000000,0.996546,0.998270,1158.0
back.,1.000000,1.000000,1.000000,750.0
teardrop.,1.000000,1.000000,1.000000,712.0
warezclient.,1.000000,1.000000,1.000000,667.0


# Checking on test data split....  25 %

In [50]:
testfile = 'test'
test_size = file_len(testfile)
print('Testing data has',test_size, 'observations')

Testing data has 1224630 observations


In [51]:
batchdf = pd.read_csv('test')

In [52]:
labels = batchdf[batchdf.columns[-1]].unique()
labels

array(['normal.', 'buffer_overflow.', 'loadmodule.', 'perl.', 'neptune.', 'smurf.', 'guess_passwd.', 'pod.', 'teardrop.', 'portsweep.', 'ipsweep.', 'land.', 'ftp_write.', 'back.', 'imap.', 'satan.',
       'phf.', 'nmap.', 'multihop.', 'warezmaster.', 'warezclient.', 'spy.', 'rootkit.'], dtype=object)

In [53]:
batchdf[10:test_size].shape

(1224619, 42)

In [63]:
batcharr = np.load('test_red.npy')

In [64]:
X = batcharr[:,:-1]
Y = batcharr[:,-1]

In [54]:
Y_num = np.ones(Y.shape)
Y_num[Y == 'normal.\n'] = 0
Y_cat = Y_num.copy()
Y_type = Y_num.copy()
for i in label_list['type']:
    Y_cat[Y == i+str('.\n')] = label_list[label_list['type'] == i]['id_cat']
    Y_type[Y == i+str('.\n')] = label_list[label_list['type'] == i]['id_type']

Y_num = np.ones(Y.shape)
Y_num[Y == 'normal.\n'] = 0
Y_cat = Y_num.copy()
Y_type = Y_num.copy()

for i in label_list['type']:
    Y_cat[Y == i+str('.\n')] = label_list[label_list['type'] == i]['id_cat']
    Y_type[Y == i+str('.\n')] = label_list[label_list['type'] == i]['id_type']

Y_num_hot = one_hot(Y_num.astype(int), 2)
Y_cat_hot = one_hot(Y_cat.astype(int), 5)
Y_type_hot = one_hot(Y_type.astype(int), 23)

X_new_hot = np.delete(X,[1,2,3],1)

X_new_hot2 = X[:,[1,2,3]]

# X_new.shape

model_pro_le = LE()
model_pro_ohe = OHE(handle_unknown='ignore')
model_srv_le = LE()
model_srv_ohe = OHE(handle_unknown='ignore')
model_flg_le = LE()
model_flg_ohe = OHE(handle_unknown='ignore')

model_pro_ohe.fit_transform(model_pro_le.fit_transform(cat_uniqs[0]).reshape(-1,1))
model_srv_ohe.fit_transform(model_srv_le.fit_transform(cat_uniqs[1]).reshape(-1,1))
model_flg_ohe.fit_transform(model_flg_le.fit_transform(cat_uniqs[2]).reshape(-1,1))

X_pro = model_pro_ohe.transform(model_pro_le.transform(X_new_hot2[:,0]).reshape(-1,1)).todense()
X_srv = model_srv_ohe.transform(model_srv_le.transform(X_new_hot2[:,1]).reshape(-1,1)).todense()
X_flg = model_flg_ohe.transform(model_flg_le.transform(X_new_hot2[:,2]).reshape(-1,1)).todense()

# X_srv.shape

X_new = np.concatenate((X_new_hot , X_pro , X_srv, X_flg), axis = 1).astype(float)

X_new.shape

In [67]:
model.score(X_new,Y)

0.9997863805003094

In [55]:
Y_pr = model.predict(X_new)

In [56]:
np.set_printoptions(linewidth = 200)
print(cmat(Y, Y_pr))

[[   303      0      0      0      0      0      0      0      0      0      0      2      0      0      0      0      0      0      0      0      0      0      0]
 [     0      5      0      0      0      0      0      1      0      0      0      1      0      0      0      0      0      0      0      0      0      0      0]
 [     0      0      1      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0]
 [     0      0      0     16      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0      0]
 [     0      0      0      0      3      0      0      0      0      1      0      0      0      0      0      0      0      0      0      0      0      0      0]
 [     0      0      0      0      0   1327      0      0      0      0      2      1      0      0      0      0      0      0      0      0      1      0      0]
 [     0      0 

In [61]:
Y_pr.shape

(318323,)

In [68]:
prfsarr = np.array(prfs(Y, Y_pr, labels = labels))
df = pd.DataFrame(prfsarr.T, columns = ['precison', 'recall', 'f_score', 'support'], index = labels)
df

/home/abhikcr/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


,precison,recall,f_score,support
normal.,0.999902,0.999892,0.999897,213737.0
buffer_overflow.,0.714286,0.714286,0.714286,7.0
loadmodule.,0.333333,0.250000,0.285714,4.0
perl.,1.000000,1.000000,1.000000,1.0
neptune.,0.999969,0.999980,0.999975,98058.0
smurf.,1.000000,1.000000,1.000000,1195.0
guess_passwd.,1.000000,1.000000,1.000000,16.0
pod.,1.000000,1.000000,1.000000,57.0
teardrop.,0.995690,1.000000,0.997840,231.0
portsweep.,0.996910,0.992821,0.994861,975.0


In [69]:
df.mean()

precison        0.804184
recall          0.834030
f_score         0.806702
support     13840.130435
dtype: float64

### Thankyou for viewing